**Task 07: Querying RDF(s)**

In [14]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Read the RDF file

In [15]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N9f55a666e6934ff59ce10fc0978655a4 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [22]:
# 7.1 RFDLib 
ns = Namespace("http://somewhere#")
search = g.triples((None, RDFS.subClassOf, ns.Person))

for subj, pred, obj in search:
  print(subj)


http://somewhere#Researcher


In [23]:
# 7.1 SPARKQL

from rdflib.plugins.sparql import prepareQuery

search = prepareQuery('''
  SELECT 
    ?Subject
  WHERE { 
    ?Subject rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)


for res in g.query(search):
  print(res)

(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [25]:
# 7.2 RFDLib 
searchInPerson = g.triples((None, RDF.type, ns.Person))
searchSubPerson = g.triples((None, RDFS.subClassOf, ns.Person))
for subj, pred, obj in searchInPerson:
  print(subj)

for subj, pred, obj in searchSubPerson:
  for inner_subj, inner_pred, inner_obj in g.triples((None, RDF.type, subj)):
    print(inner_subj)

http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith


In [24]:
# 7.2 SPARKQL
search = prepareQuery('''
    SELECT 
        ?Subject
      WHERE { 
        ?Subject rdf:type/rdfs:subClassOf* ns:Person. 
      }
    ''',
    initNs={"ns": ns}
)

for res in g.query(search):
    print(res)

(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [26]:
# 7.3 RFDLib 
for subj, pred, obj in g.triples((None, RDF.type, ns.Person)):
  for inner_subj, inner_pred, inner_obj in g.triples((subj, None, None)):
    print(inner_subj, inner_pred, inner_obj)
for subj, pred, obj in g.triples((None, RDFS.subClassOf, ns.Person)):
  for inner_subj, inner_pred, inner_obj in g.triples((None, RDF.type, subj)):
    for sec_inner_subj, sec_inner_pred, sec_inner_obj in g.triples((inner_subj, None, None)):
      print(sec_inner_subj, sec_inner_pred, sec_inner_obj)

http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewher

In [27]:
# 7.3 SPARKQL
search = prepareQuery('''
SELECT ?subject ?predictor ?object
WHERE{
  ?subject rdf:type/rdfs:subClassOf* ns:Person.
?subject ?predictor ?object
}
''',
initNs = {"ns": ns}
)
for res in g.query(search):
  print(res)

(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://somewhere#Person'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.Literal('John Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.Literal('John', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.Literal('Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/